In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.stats import gmean
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models, expected_returns, plotting
from pandas.plotting import scatter_matrix

In [23]:
class PortfolioOptimiser:

    def __init__(self, tickers, time_period = 365):
        "Loading Basic Returns of Portfolio"
        self.time_period = time_period
        self.end_date = datetime.today()
        self.start_date = self.end_date - timedelta(days=time_period)
        self.tickers = [ticker.upper().strip() for ticker in tickers]
        data = yf.download(self.tickers, start = self.start_date, end = self.end_date, auto_adjust = False, progress = False, threads = False)
        self.prices = data['Adj Close']
        self.returns = self.prices.pct_change().dropna()
    
    def get_rf_rate(self):
        rate = float((yf.download('^TNX', period='1d',auto_adjust = False, progress = False)['Close'].iloc[-1] / 100).iloc[0])
        self.rf_return = rate
        return self.rf_return
        
    def print_rf_rate(self):
        print(f"The risk free rate is: {self.get_rf_rate() * 100}%")
    
    def individual_metrics(self):
        "Calculating key metrics for each asset"
        self.get_rf_rate()
        results = []
        for ticker in self.tickers:
            ticker_returns = self.returns[ticker]
            daily_mean = float(ticker_returns.mean())
            daily_std = float(ticker_returns.std(ddof=1))
            geo_annual_return = float((1 + daily_mean) ** 252 - 1)
            annual_vol = float(daily_std * np.sqrt(252))
            excess_return = geo_annual_return - self.rf_return
            sharpe_ratio = float(excess_return/annual_vol)
            if annual_vol < 0.05:
                risk = 'Low Risk'
            elif annual_vol >= 0.05 and annual_vol < 0.15:
                risk = 'Moderate Risk'
            elif annual_vol >= 0.15:
                risk = 'High Risk'
            results.append({
                'Ticker': ticker,
                'Annual Return (%)': geo_annual_return * 100,
                'Annual Volatility (%)': annual_vol * 100,
                'Excess Return (%)': excess_return * 100,
                'Sharpe Ratio': sharpe_ratio,
                'Risk Category': risk})
        return pd.DataFrame(results)

    def covariance_matrix(self):
        "Calculating Covariance Matrix"
        cov_matrix = self.returns.cov(numeric_only = True) *252
        return cov_matrix

    def correlation_matrix(self):
        corr_matrix = self.returns.corr(numeric_only = True)
        return corr_matrix

    def plotting_sm(self):
        sm = scatter_matrix(self.returns, figsize = (10,10))
        plt.show()
        return sm

    def portfolio_optimisation_sharpe(self):
        mu = self.returns.mean() * 252
        S = self.returns.cov() * 252
        results = {}
        
        ef_sharpe = EfficientFrontier(mu, S, weight_bounds=(0, 1))
        weights_sharpe = ef_sharpe.max_sharpe()
        perf_sharpe = ef_sharpe.portfolio_performance(verbose=False)
        results['max_sharpe'] = {'weights': ef_sharpe.clean_weights(),
        'return': perf_sharpe[0],
        'volatility': perf_sharpe[1], 
        'sharpe': perf_sharpe[2]}
        return results

    
    def plot_sharpe(self):
        mu = self.returns.mean() * 252
        S = self.returns.cov() * 252

        "Plotting the Efficient Frontier"
        ef_plot = EfficientFrontier(mu, S, weight_bounds=(0,1))
        plotting.plot_efficient_frontier(ef_plot, show_assets = True)
        plt.show()

    def portfolio_optimisation_vol(self):
        mu = self.returns.mean() * 252
        S = self.returns.cov() * 252
        results = {}
        
        ef_vol = EfficientFrontier(mu, S, weight_bounds=(0, 1))
        weights_vol = ef_vol.min_volatility()
        perf_vol = ef_vol.portfolio_performance(verbose=False)
        results['min_volatility'] = {
        'weights': ef_vol.clean_weights(),
        'return': perf_vol[0],
        'volatility': perf_vol[1],
        'sharpe': perf_vol[2]}
        return results
        
    def plot_vol(self):
        mu = self.returns.mean() * 252
        S = self.returns.cov() * 252

        ef_plot = EfficientFrontier(mu, S, weight_bounds = (0,1))
        plotting.plot_efficient_frontier(ef_plot, show_assets = True)
        plt.show()
        

    def export_to_excel(self, filename):
        if not filename.endswith('.xlsx'):
            filename += '.xlsx'
        
        individual_df = self.individual_metrics()
        cov_df = self.covariance_matrix()
        corr_df = self.correlation_matrix()
        opt_results_1 = self.portfolio_optimisation_sharpe()
        opt_results_2 = self.portfolio_optimisation_vol()
        opt_df_1 = pd.DataFrame.from_dict(opt_results_1, orient='index')
        opt_df_2 = pd.DataFrame.from_dict(opt_results_2, orient='index')
        combined_results = {**opt_results_1, **opt_results_2}
        #opt_df = pd.json_normalize(combined_results).T
        opt_df = pd.DataFrame(combined_results).T
        with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
            individual_df.to_excel(writer, sheet_name = 'Analysis', index = False, startrow = 0, startcol = 0)
            cov_df.to_excel(writer, sheet_name = 'Analysis', index = False, startrow = 0, startcol = 1 + len(individual_df.columns))
            corr_df.to_excel(writer, sheet_name = 'Analysis', index = False, startrow = 2 + len(cov_df), startcol = 1 + len(individual_df.columns))
            opt_df.to_excel(writer, sheet_name = 'Analysis', index = False, startrow = 2 + len(individual_df), startcol = 0)
            #opt_df_2.to_excel(writer, sheet_name = 'Analysis', index = False, startrow = 20, startcol = 0)